# Mamy model i co dalej?

Przygotowaliśmy dane, wybralismy cechy, wybraliśmy model - gotowe! No, nie do końca. Jest ML żeby wygrać w buzzwords bingo potrzebujemy jeszcze microservice'u! ;-)

In [6]:
import pandas as pd
import sklearn.model_selection
import sklearn.linear_model
import sklearn.metrics
import seaborn as sns
import math
%matplotlib inline  

r = pd.read_csv('ceny_mieszkan_w_poznaniu.tsv', sep = '\t')

r_train, r_test = sklearn.model_selection.train_test_split(r, test_size = 0.2)
model = sklearn.linear_model.LinearRegression()

features = ['sqrMeters', 'rooms']
label = ['price']

model = sklearn.linear_model.LinearRegression()
X_train = r_train[features]
y_train = r_train[label].values.reshape(-1, 1)

X_test = r_test[features]
y_test = r_test[label].values.reshape(-1, 1)
model.fit(X_train,y_train)

sqr_meters = 71
no_of_rooms = 2
model.predict([[sqr_meters, no_of_rooms]])

array([[328836.65397741]])

W przypadku sklearn'a można wyeksportować model w formie pliku *pickle*, który następnie można wczytać w miejscu gdzie chcemy z niego korzystać. 

In [7]:
import pickle

# model zostanie zapisany w pliku pkl
pickle.dump(model, open("model.pkl","wb"))

# możemy go potem wczytać i wykorzystać do serwowania predykcji
model_p = pickle.load(open("model.pkl","rb"))

model_p.predict([[sqr_meters, no_of_rooms]])

array([[328836.65397741]])

Przykład osadzenia takiego modelu w prostej aplikacji webowej możecie znaleźć w podkatalogu `ai_service`. 

In [8]:
!ls ai_service/version_01

__pycache__   ai_service.py model.pkl     readme.txt


In [9]:
!cat ai_service/version_01/ai_service.py

import pickle
import flask
from flask import request

app = flask.Flask(__name__)

#getting our trained model from a file we created earlier
model = pickle.load(open("model.pkl","rb"))

@app.route('/predict', methods=['POST'])
def predict():
    #grabbing a set of features from the request's body
    feature_array = request.get_json()['feature_array']
    
    #our model rates flat based on the input array
    prediction = model.predict([feature_array]).tolist()
    
    #preparing a response object and storing the model's predictions
    response = {}
    response['predictions'] = prediction
    
    #sending our response object back as json
    return flask.jsonify(response)

Informacje o tym jak uruchomić aplikacje możecie znaleźć w pliku readme.txt

In [10]:
!cat ai_service/version_01/readme.txt

Install flask: 
pip3 install flask

Launch the app: 
export FLASK_APP=ai_service.py; flask run

Get predictions: 
curl 'http://localhost:5000/predict' -d '{"feature_array":[71,3]} ' -XPOST -H "Content-type: application/json"

Should result in something like:
{"predictions":[[419191.0334601513]]}



# Pipelines

Przygotowując dane na potrzeby uczenia modelu dokonujemy różnych przekształceń (dodajemy one-hot'y, przemapowujemy cechy, wybieramy tylko niektóre kolumny). Ważne, jest żeby pamiętać, że analogiczne przeszktałcenia muszą zostać przeprowadzone w przypadku danych wejściowych na etapie predykcji. Z pesperktywy praktycznej najlepiej jak ten sam kod jest używany na etapie trenowania jak i predykcji. Sklearn udostępnia Pipelines, dzięki którym możemy zapisać sekwencje operacji w formie podobnej do bash'owego potoku ;-) 

np. wybierz_cechy | dodaj cechy one hot | policz_coś_tam | wytrenuj

Przykładowy pipeline wygląda jak poniżej.

In [11]:
from sklearn.pipeline import Pipeline
from enhancer import FeatureEnhancer

regr = sklearn.linear_model.LinearRegression()
features = FeatureEnhancer()
p = Pipeline([
    ('feature selection', features),
    ('regression', regr)
])

X_train = r_train
y_train = r_train[label].values.reshape(-1, 1)

p.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('feature selection', FeatureEnhancer()), ('regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

Każdy z kroków przetwarzania to tak naprawdę obiekt, który musi implementować metody fit i transform. Powyższe wywołanie metody p.fit() spowoduje odpalenie metod transform i fit na elementach w pip'ie.

Poniżej przykład klasy FeatureEnhancer, który wybiera z podanego dataframe'a cechy istotne w naszym modelu i dodaje nową ```is_center```. 

In [12]:
!cat enhancer.py

from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEnhancer(BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):      
        
        def get_is_center(district):
            if district is "Wilda":
                return 1
            return 0
        
        out = X.copy()
        out['is_center'] = X['location'].map(get_is_center)
        return out[['sqrMeters', 'rooms','is_center']]

Przekształcenia wykonane na etapie uczenia zostaną również zaaplikowane na etapie predykcji.

In [14]:
input_df = pd.DataFrame({
              'sqrMeters' : pd.Series([120]), 
              'rooms' : pd.Series([4]),
              'location' : pd.Series(['Wilda'])
            })

p.predict(input_df)

array([[584076.05515689]])

Tak wytrenowany model wraz z przekształceniami możemy zserializować i zapisać do wykorzystania w drugiej wersji naszej aplikacji.

In [15]:
model_filename = "pipe_with_module.pkl"

pickle.dump(p, open(model_filename,"wb"))

In [16]:
piped = pickle.load(open(model_filename,"rb"))

In [17]:
piped.predict(input_df)

array([[584076.05515689]])

In [18]:
!ls ai_service/version_01

__pycache__   ai_service.py model.pkl     readme.txt


**Ex. 1**

Doprowadźcie drugą wersję ai_service do postaci działającej.

In [21]:
!cat ai_service/version_02/ai_service.py

import pickle
import flask
from flask import request
import pandas as pd
from enhancer import FeatureEnhancer
from sklearn.externals import joblib

app = flask.Flask(__name__)

#getting our trained model from a file we created earlier
model = pickle.load(open("pipe_with_module.pkl","rb"))

@app.route('/predict', methods=['POST'])
def predict():
    #grabbing a set of features from the request's body
    input_dataframe = # construct dataframe from request.get_json()
    
    #our model predicts price of flat based on dataframe
    prediction = model.predict(input_dataframe).tolist()
    
    #preparing a response object and storing the model's predictions
    response = {}
    response['prediction'] = prediction
    
    #sending our response object back as json
    return flask.jsonify(response)



In [20]:
!cat ai_service/version_02/readme.txt

Install flask: 
pip3 install flask

Launch the app: 
export FLASK_APP=ai_service.py; flask run

Get predictions: 
curl 'http://localhost:5000/predict' -d '{"sqrMeters":71, "rooms":2, "location":"Wilda"} ' -XPOST -H "Content-type: application/json"

Should result in something like:
{"prediction":[[419191.0334601513]]}

